In [3]:
import re
from pathlib import Path
import pandas as pd

FILE  = Path("C://Users//shoaib//Desktop//Mental_health_ed_project//820-hospital-stays-for-harm-caused-by-substance-use-data-table-en (1).xlsx") 
SHEET = "Table 1"                                            
OUT   = Path("clean/cihi_substance_hosp_long.csv")
OUT.parent.mkdir(exist_ok=True)

SUPPRESSION = {"–", "-", "N/A", "n/a", "Not applicable", "NR"}

KEEP_BREAKDOWN = "Not applicable"   # or "Age group", "Recorded sex or gender", ...

PROV_MAP = {  # canonical two-letter province codes
    "Newfoundland and Labrador":"NL", "Prince Edward Island":"PE", "Nova Scotia":"NS",
    "New Brunswick":"NB", "Quebec":"QC", "Ontario":"ON", "Manitoba":"MB",
    "Saskatchewan":"SK", "Alberta":"AB", "British Columbia":"BC", "Yukon":"YT",
    "Northwest Territories":"NT", "Nunavut":"NU", "Canada":"CAN"
}

df = pd.read_excel(FILE, sheet_name=SHEET, dtype=str)

# normalise headers
df.columns = [re.sub(r"[^0-9a-z]+", "_", str(c).strip().lower()).strip("_")
              for c in df.columns]

# 2 ────────────── basic cleaning ─────────────────────────────────────
df.replace(list(SUPPRESSION), pd.NA, inplace=True)

# keep only the rows you need (totals or chosen breakdown)
df = df[df["level_1_breakdown"].fillna("Not applicable").str.contains(KEEP_BREAKDOWN, case=False)]

# province codes
df["province"] = (df["province_territory"]
                  .fillna("Canada")
                  .str.strip()
                  .map(PROV_MAP)
                  .fillna(df["province_territory"].str.strip()))

# fiscal year: grab first 4-digit year in "2017–2018"
df["fiscal_year"] = (df["time_frame"]
                     .str.extract(r"(\d{4})")
                     .astype("Int16"))

# numeric casts
df["count"]      = pd.to_numeric(df["numerator"],   errors="coerce")
df["population"] = pd.to_numeric(df["denominator"], errors="coerce")
df["crude_rate"] = pd.to_numeric(df["crude_result"], errors="coerce")

tidy = df[["province", "fiscal_year", "count", "population", "crude_rate"]]

dups = tidy.duplicated(["province", "fiscal_year"]).sum()
if dups:
    print(f"{dups} duplicate province-year rows (check breakdown filter)")

# 4 ────────────── save ───────────────────────────────────────────────
tidy.to_csv(OUT, index=False)
print(f"Clean file → {OUT}   rows={len(tidy):,}")


198 duplicate province-year rows (check breakdown filter)
Clean file → clean\cihi_substance_hosp_long.csv   rows=296


In [5]:
import pandas as pd

PATH_IN  = "clean/cihi_substance_hosp_long.csv"
PATH_OUT = "clean/cihi_substance_hosp_final.csv"

df = pd.read_csv(PATH_IN)

# 1. drop banner 
df = df[df["fiscal_year"].notna()].copy()

# 2. cast fiscal_year
df["fiscal_year"] = df["fiscal_year"].astype("int16")

# 3. collapse duplicates (province-year)
agg = (df.groupby(["province", "fiscal_year"], as_index=False)
         .agg({"count":"sum", "population":"sum"}))

# 4. derive crude rate per 100 000
agg["crude_rate"] = agg["count"] / agg["population"] * 100_000

# 5. flag rows still missing counts/pop
agg["was_suppressed"] = agg["count"].isna() | agg["population"].isna()

# 6. save
agg.to_csv(PATH_OUT, index=False)
print(f"final tidy file → {PATH_OUT}   rows={len(agg):,}")


final tidy file → clean/cihi_substance_hosp_final.csv   rows=97
